# SS記事の自動カテゴリ分類

運営しているSSまとめサイトで、それぞれのSSのカテゴリ付けを自動化できないか検証してみた。

使用したデータは、DBにクエリを投げてダンプしたもの。

このノートブックでは、それぞれの記事の頻出名詞top10を特徴量として分類する。

### 環境

- Ubuntu 14.04
- python3.5

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import multiprocessing
from collections import Counter
import re

from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
# pandas.DataFrame.apply関数の並列版
def applyParallel(dfGrouped, func, asDataFrame=False):
    retLst = Parallel(n_jobs=multiprocessing.cpu_count())(delayed(func)(field) for field in dfGrouped)
    if asDataFrame:
        return pd.DataFrame(retLst)
    else:
        return pd.Series(retLst)

## データ理解

データの読み込み

In [3]:
# セパレータがカンマだとうまく読み込めなかったので、独自セパレータを使用
df = pd.read_csv('./articles.csv', sep='PNDFAKFD', encoding='utf-8')

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [4]:
df[:10]

,article_id,category_id,category_name,html
0,1,3.0,艦これ,<dd>提督「(俺はこの鎮守府の提督)」 <br> <br> 提督「(深海棲艦という、謎の...
1,2,3.0,艦これ,<dd>時雨「今日は雨だね、提督」 <br> <br> 時雨「そういえば」 <br> <...
2,3,NaN,NaN,<dd>女騎士「何！？オークとオーガは似たような種族ではないのか！？」 <br> <br>...
3,4,2.0,男女,<dd> 幼「約束は８時でしょ？」<br><br>男「悪かったよ、ちょっと仕事がな…」<br...
4,5,11.0,化物語,<dd> <br> ・アイドルマスター・ディアリースターズと物語シリーズのクロスです。 <b...
5,6,NaN,NaN,<dd> <br> メリオダス・ギル坊「じゃんけんほい！！！」 <br> <br> <b...
6,7,NaN,NaN,<dd> <br> <br> <br> <br> <br> ｻﾞｧｰｰｰｰｰｰｰ･･...
7,8,10.0,モバマス,<dd>秋風温泉ガチャをネタ元にしたＳＳです <br> 内容はぶっちゃけ蘭子かな子芳乃とのハ...
8,9,10.0,モバマス,<dd>冬ということは忘れてください <br> <br> <br> <br> 「「「た...
9,10,NaN,NaN,"<dd class=""net""> 俺「よろしく！うわぁー、ワクワクするなぁ！」 <br> <..."


カテゴリIDとカテゴリ名の対応付け

In [5]:
tmp_df = df[['category_id', 'category_name']].dropna()
tmp_df.category_id = tmp_df.category_id.astype(int)

category_df = tmp_df.groupby(['category_id']).agg(lambda x:x.value_counts().index[0])
category_df[:5]

,category_name
category_id,
1,俺妹
2,男女
3,艦これ
4,とある魔術の禁書目録
5,兄妹・姉弟


In [6]:
count_each_category = df[['article_id', 'category_name']].groupby(['category_name']).agg('count').sort_values('article_id', ascending=False)
count_each_category[:15]

,article_id
category_name,
モバマス,4586
艦これ,1787
アイドルマスター,1642
俺ガイル,953
ラブライブ！,725
男女,695
シュタインズ・ゲート,461
勇者・魔王,388
咲,308


モバマス・艦これが多い

## データ整形

まずは記事数が多くて、かつ分類しやすそうなカテゴリだけ選んで試してみる。

(ex. 「男女」と「兄妹・姉弟」などは分類しにくそうなので除外）

In [7]:
easy_categories = ['モバマス', '艦これ', 'アイドルマスター', '俺ガイル', 'ラブライブ！', 'シュタインズ・ゲート', '咲', 'とある魔術の禁書目録', 'ゆるゆり', 'ガールズ＆パンツァー']
easy_df = df[df.category_name.isin(easy_categories)]

print('カテゴリー数: {}'.format(len(easy_categories)))
print('対象記事数: {}'.format(easy_df.shape[0]))

カテゴリー数: 10
対象記事数: 11339


BeautifulSoupでhtmlタグとurlを除去。
8コアをフルに使っても2分くらいかかる。

In [8]:
def extract_text(html):
    text = BeautifulSoup(html, "html.parser").getText() # htmlタグを除去
    return re.sub(r'https?://[^\s/$.?#].[^\s]*', '', text) # urlを除去

easy_df['text'] = applyParallel(easy_df.html, extract_text).values

easy_df = easy_df.drop('html', axis=1)
easy_df.category_id = easy_df.category_id.astype(int)
easy_df[:10]

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,article_id,category_id,category_name,text
0,1,3,艦これ,提督「(俺はこの鎮守府の提督)」 提督「(深海棲艦という、謎の敵から海を守るため)」 ...
1,2,3,艦これ,時雨「今日は雨だね、提督」 時雨「そういえば」 時雨「レイテ沖のときも雨が降っていた...
7,8,10,モバマス,秋風温泉ガチャをネタ元にしたＳＳです 内容はぶっちゃけ蘭子かな子芳乃とのハーレムエロなんで...
8,9,10,モバマス,冬ということは忘れてください 「「「ただいまー！！」」」 P「おう みんなおかえ...
11,12,3,艦これ,提督「珍しいな、溜息なんてついて」 那珂「あっ、提督！ たたた溜息なんてついてないよ！？...
12,13,9,アイドルマスター,＿＿＿＿＿＿＿＿＿＿＿＿７６５プロ 事務所 仕事終わりの事務所にやよい、真、響、春香、美...
15,16,3,艦これ,【加賀の場合】 「提督、失礼します」 「あら……寝ているの？」 「最近忙しか...
18,19,3,艦これ,提督の第一印象？ ……提督だとは思わなかった、ですね。 ふざけた芸人...
19,20,3,艦これ,提督「(雲龍がこの鎮守府に来てくれて)」 提督「(しばらく経った)」 提督「(今では...
20,21,9,アイドルマスター,"""5"" ""4"" 千早「プロデューサー、いよいよですね」 ""3"" ..."


## 特徴量抽出

SSは以下のような台本形式のものが多いので、頻出する名詞を特徴量とすればうまく分類できるはず。

```
Ｐ「それがよくわからないんだよ……おいしいのかな？」 

凛「私に聞かれても……」 

ちひろ「まぁまぁまぁ、そんなことより凛ちゃんはこれからレッスンですよね？」
```

In [9]:
import MeCab
tagger = MeCab.Tagger('-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse('') #　おまじない(mecabのバグ)

'EOS\n'

Nelogd辞書がちゃんと機能しているかチェック

In [10]:
node = tagger.parseToNode("新辞書が必要なのは、なのはみたいなややこしい名前が入ってるとき")
node = node.next

while node and node.surface:
    print("{}\t\t{}".format(node.surface, node.feature))
    node = node.next

新		接頭詞,名詞接続,*,*,*,*,新,シン,シン
辞書		名詞,一般,*,*,*,*,辞書,ジショ,ジショ
が		助詞,格助詞,一般,*,*,*,が,ガ,ガ
必要		名詞,形容動詞語幹,*,*,*,*,必要,ヒツヨウ,ヒツヨー
な		助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
の		名詞,非自立,一般,*,*,*,の,ノ,ノ
は		助詞,係助詞,*,*,*,*,は,ハ,ワ
、		記号,読点,*,*,*,*,、,、,、
なのは		名詞,固有名詞,人名,一般,*,*,なのは,ナノハ,ナノハ
みたい		名詞,非自立,形容動詞語幹,*,*,*,みたい,ミタイ,ミタイ
な		助動詞,*,*,*,特殊・ダ,体言接続,だ,ナ,ナ
ややこしい		形容詞,自立,*,*,形容詞・イ段,基本形,ややこしい,ヤヤコシイ,ヤヤコシイ
名前		名詞,一般,*,*,*,*,名前,ナマエ,ナマエ
が		助詞,格助詞,一般,*,*,*,が,ガ,ガ
入っ		動詞,自立,*,*,五段・ラ行,連用タ接続,入る,ハイッ,ハイッ
てる		動詞,非自立,*,*,一段,基本形,てる,テル,テル
とき		名詞,非自立,副詞可能,*,*,*,とき,トキ,トキ


それぞれの記事の頻出名詞TOP10を抽出

In [11]:
def get_word_counter(text):
    word_array = []
    
    node = tagger.parseToNode(text)
    while node:
        split = node.feature.split(',')
        if split[0] == '名詞' and (split[1] in ['一般', '固有名詞']):
            word_array.append(node.surface)
        node = node.next
    return Counter(word_array)

def get_feature_words(text):
    common_words_and_count = get_word_counter(text).most_common(10)
    return list(map(lambda e: e[0], common_words_and_count))

for _, row in easy_df[10:15].iterrows():
    print(row.category_name)
    print(get_feature_words(row.text))
    print()

艦これ
['提督', '鳳翔', '鳳', '翔', 'あと', '店', 'ｺﾞﾄﾝ', '雪', 'サナトリウム', '鎌倉']

モバマス
['千秋', '智香', 'P', '力', 'ﾊﾞｯ', '千夏', 'アタシ', 'ｼﾞｰ', 'ー', '動き']

アイドルマスター
['千早', '黒井', '冬馬', 'カレー', 'ウィ', 'ー', '社長', 'あずさ', 'では', 'プロ']

艦これ
['提督', '曙', 'ー', 'レ', '扶桑', 'うむ', '木曾', 'ﾄﾞﾝ', '部屋', '味噌']

とある魔術の禁書目録
['上条', '土御門', 'カミ', 'オティヌス', '禁書', '美琴', 'アリサ', 'シャットアウラ', '裂', 'レッサー']



8コアで2分くらいかかる

In [12]:
easy_df['feature_words'] = applyParallel(easy_df.text, get_feature_words).values
easy_df[:10][['category_name', 'feature_words']]

,category_name,feature_words
0,艦これ,"[提督, 翔鶴, 憲兵, 鳳翔, ー, ふふ, 艦娘, 剛, 金, 一緒に]"
1,艦これ,"[扶桑, 榛名, 提督, 雷, 時雨, 姉, 雪風, 五月雨, 加賀, 曙]"
7,モバマス,"[Ｐ, 蘭子, かな子, 芳, 乃, ♥, ー, ｽﾞﾝｯ, そなた, ﾋﾞｸﾝｯ]"
8,モバマス,"[千秋, ザリガニ, P, ｸﾜｧ, 薫, 千佳, 仁, 奈, ー, 菲菲]"
11,艦これ,"[那珂, 提督, 野分, アイドル, 三日月, 扶桑, ちゃ, ふれ, 溜息, 娘]"
12,アイドルマスター,"[真, 響, 春香, P, 美希, チョコ, イカ, やよい, プロデューサー, トマト]"
15,艦これ,"[提督, 司令官, ゴーヤ, イムヤ, ハチ, イク, 北上, 青葉, ー, カレー]"
18,艦これ,"[提督, 赤城, 叢雲, 好きだ。, 鎮守府, 人間, 敵, 鈴谷, 加古, 大井]"
19,艦これ,"[提督, 雲龍, ｶｷｶｷ, 天城, ﾆｺ, ふふ, ぅ, ぇ, ﾁﾗ, ｽｯ]"
20,アイドルマスター,"[千早, P, 春香, プロデューサー, アイドル, 律子, 伊織, 月, 理想像, 社長]"


単語 -> インデックスのdictionaryを作成

In [13]:
def flatten(l):
    return [item for sublist in l for item in sublist]

word2index = {}
for i,v in enumerate(set(flatten(easy_df.feature_words.values))):
    word2index[v] = i

num_words = len(word2index)
"辞書内の単語数: {}".format(num_words)

'辞書内の単語数: 10945'

In [14]:
def sparse_to_dense(sparse, length):
    dense = np.zeros(length, dtype=np.int32)
    for i in sparse:
        dense[i] = 1
    assert len(dense) == length
    return dense

def get_dense_features(feature_words):
    global num_words, word2index
    sparse = map(lambda x: word2index[x], feature_words)
    return sparse_to_dense(sparse, num_words)

easy_df[:3].feature_words.apply(get_dense_features)

0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
7    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: feature_words, dtype: object

特徴量行列を作成

In [15]:
features = applyParallel(easy_df.feature_words, get_dense_features, asDataFrame=True)
labels = easy_df.category_id

## scikit-learnで予測

いくつかの手法で予測。

- SVM
- Logistic Regression
- Random Forest

データがそこそこ大きいので、解析的に解くモデルだと終わりません。

In [16]:
# テンプレート
def evaluate_model(model):
    global features, labels
    scores = cross_validation.cross_val_score(model, features, labels, cv=5)
    print("Accuracy: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

### クロスバリデーションでモデルを評価

In [17]:
# SVM
evaluate_model(SGDClassifier(loss="hinge", penalty='l2', n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.97733 (+/- 0.00698)


In [18]:
# ロジスティック回帰
evaluate_model(SGDClassifier(loss="log", penalty='l2', n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.97636 (+/- 0.00769)


In [19]:
# ランダムフォレスト
evaluate_model(RandomForestClassifier(n_estimators=10, n_jobs=multiprocessing.cpu_count()))

Accuracy: 0.97531 (+/- 0.00774)


だいたい全部同じくらいの精度

以下SVMを使用

In [20]:
split_point = int(len(labels) * 0.8)
train_features = features[:split_point]
test_features = features[split_point:]
train_labels = labels[:split_point]
test_labels = labels[split_point:]

pred = SGDClassifier(loss="hinge", penalty='l2', n_jobs=multiprocessing.cpu_count()).fit(train_features, train_labels).predict(test_features)
pd.crosstab(pred, test_labels)

category_id,3,4,6,9,10,12,20,26,27,35
row_0,,,,,,,,,,
3,265,1,0,0,3,0,0,0,0,1
4,0,54,1,1,1,1,0,0,0,0
6,0,0,198,0,2,0,0,0,0,0
9,0,0,0,233,5,0,0,0,0,0
10,3,0,2,11,1069,0,1,1,1,2
12,0,0,0,0,0,95,0,0,0,0
20,0,0,0,0,0,0,60,0,0,0
26,0,0,0,0,2,0,0,49,0,0
27,0,0,2,2,4,0,0,1,40,1


分類に失敗したSSを眺めてみる

In [21]:
test_df = easy_df.ix[test_labels.index]
test_df['pred'] = category_df.category_name[pred].values

tmp = test_labels != pred
test_df.ix[tmp[tmp == True].index][['article_id', 'category_name', 'pred', 'feature_words']]

,article_id,category_name,pred,feature_words
15080,15716,ラブライブ！,とある魔術の禁書目録,"[ことり, 講師, 人, 顔, デザイナー, 老人, 服, 少女, 道, 一つ]"
15105,15741,モバマス,とある魔術の禁書目録,"[早苗, 夏美, ゴヨウ, 幻獣, トークン, エンド, 特殊召喚, バトル, エンタープラ..."
15458,16094,アイドルマスター,とある魔術の禁書目録,"[ヤンキー, 頭, 自分, ォ, ス, ワケ, 可奈, はるるん, ー, アリーナ]"
15629,16265,咲,モバマス,"[佑子, 大星, マユ, …。, イ, 顔, 身体, 全身, 乳首, ぁっ]"
15659,16295,アイドルマスター,モバマス,"[猫, おじ, ー, 漱石, 手, アンマー, ぃに, テレビ, 部屋, 下]"
15697,16333,モバマス,艦これ,"[場所, 少女, 空間, 姿, トモダチ, 声, 机, 無機質, 音, 顔]"
15830,16466,モバマス,咲,"[愛海, 巴, わき, サラシ, くるみちゃん, くるみ, お山, ぇ, うち, ふぇぇ]"
15871,16507,モバマス,アイドルマスター,"[Ｐ, ちひろ, 文香, 晴, 真美, プロデューサー, 響, モバ, 春香, 美希]"
15889,16525,モバマス,ラブライブ！,"[ほんとう, 一言, うそ, エイプリルフール, 想い, 店, 貴女, クリ, 場所, 初の]"
15904,16540,艦これ,モバマス,"[卯月, 加古, 嘘, エイプリルフール, 古鷹, 出て, ω, 布団, はいはい, ろん]"


## 考察

分類に失敗したものには、以下のパターンがあるようだ。（番号はarticle_id）

1. そもそも正解ラベルが間違っている（だいたいアイマス <-> モバマス）
  - 16632
  - 17270
  - 17413
  - 17462
  - 17689
  - 17770
  - 17901
  - 17925
  - 19022
  - 19308
2. 正解が曖昧（複数作品のクロスものなど）
  - 16594
  - 17501
  - 18688
  - 19097
3. 名前が似ているキャラクター
  - 16540
  - 17450
  - 19018
4. 台本形式じゃないため、名前がうまく取得できなかった
  - 15716
  - 16142
  - 16265
  - 16295
  - 16525
  - 17648
  - 18708

#### 思ったこと

- 正解ラベル間違いすぎぃ！
- 3,4の理由のものは頑張って正しく分類できるようになりたい
- 予想どおり台本形式じゃないSSは分類しにくい

## まとめ

- だいたい97.5%くらいの正答率で分類できたよ！（人の方が精度は上）
- 正解ラベルが正しければ、99%くらいの精度かな？
- 特徴量は頻出する名詞（キャラクターの名前）を利用
- 台本形式じゃないSSはうまく分類できない
- ニューラルネットとか使おうかと思ってたけど、そこまでは必要なさそう

## これから

- 特徴量の作り方次第でもうちょっと精度あげれそう
- TD-IDFあたりの考え方を使うべきか
- 本番サービスにも組み込むかも...？